# Model Class

In [95]:
class CarbonFootprintPredictor:
    def __init__(self):
        self.params = {
            'carbon_pijar': 0.02150,
            'carbon_neon': 0.00540,
            'carbon_led': 0.00240,
            'carbon_ac_non_inverter': 0.10396,
            'carbon_kulkas_inverter': 0.06500,
            'carbon_kulkas_non_inverter': 0.06600
        }

    def electric(self, daya_listrik,kulkas_inverter, kulkas_num, kulkas_consume_hour, kulkas_power, ac_inverter, ac_num, ac_consume_hour, ac_power, lamp_type, lamp_num, lamp_consume_hour, lamp_power):
        power = (kulkas_num * kulkas_consume_hour * kulkas_power) + (ac_num * ac_consume_hour * ac_power) + (lamp_num * lamp_consume_hour * lamp_power)
        return (power*30) / 1000

    def price(self, daya_listrik,kulkas_inverter, kulkas_num, kulkas_consume_hour, kulkas_power, ac_inverter, ac_num, ac_consume_hour, ac_power, lamp_type, lamp_num, lamp_consume_hour, lamp_power):
        power_kwh = self.electric(daya_listrik, kulkas_inverter, kulkas_num, kulkas_consume_hour, kulkas_power, ac_inverter, ac_num, ac_consume_hour, ac_power, lamp_type, lamp_num, lamp_consume_hour, lamp_power)

        if (daya_listrik == 450):
            power_kwh *= 415
        elif (daya_listrik == 900):
            power_kwh *= 1352
        elif (daya_listrik == 1300):
            power_kwh *= 1444
        elif (daya_listrik == 2200):
            power_kwh *= 1444
        else:
            power_kwh *= 1699
        
        return power_kwh

    def calculate_carbon(self, daya_listrik, kulkas_inverter, kulkas_num, kulkas_consume_hour, kulkas_power, ac_inverter, ac_num, ac_consume_hour, ac_power, lamp_type, lamp_num, lamp_consume_hour, lamp_power):
        electric_carbon = self.electric(daya_listrik, kulkas_inverter, kulkas_num, kulkas_consume_hour, kulkas_power, ac_inverter, ac_num, ac_consume_hour, ac_power, lamp_type, lamp_num, lamp_consume_hour, lamp_power) * 0.0094

        lamp_carbon = self.params[f'carbon_{lamp_type}'] * lamp_consume_hour * lamp_num

        ac_carbon = ac_num * ac_consume_hour
        if ac_inverter == "belum":
            ac_carbon *= self.params['carbon_ac_non_inverter']

        kulkas_carbon = (kulkas_num / 24) * kulkas_consume_hour
        if kulkas_inverter == "sudah":
            kulkas_carbon *= self.params['carbon_kulkas_inverter']
        else:
            kulkas_carbon *= self.params['carbon_kulkas_non_inverter']

        carbon = electric_carbon + lamp_carbon + ac_carbon + kulkas_carbon
        carbon *= 1000
        rounded_carbon_percentage = round((carbon / 12), 2)
        return rounded_carbon_percentage
    
    def predict_co2(self, data):
        return self.calculate_carbon(**data)
    def predict_kwh(self, data):
        return self.electric(**data)
    def predict_price(self, data):
        return self.price(**data)

In [93]:
import joblib
model = CarbonFootprintPredictor()
joblib.dump(model, "ModelPrediksi.joblib")

['ModelPrediksi.joblib']

In [22]:
cf = model.predict_co2(data.iloc[0])
print(f"Estimated monthly carbon footprint: {cf} kg CO2 equivalent")

Estimated monthly carbon footprint: 464.02 kg CO2 equivalent


In [23]:
kwh = model.predict_kwh(data.iloc[0])
print(f"Estimated monthly kwh: {kwh} kwh")

Estimated monthly kwh: 24.84 kwh


In [65]:
prc = model.predict_price(data.iloc[0])
print(f"Estimated monthly price: {prc}")

Estimated monthly price: 10308.6
